# Team Members
- Sam Ruben Abraham – 2023BCD0002
- Vinayak Sharma – 2023BCS0002


# Currency Arbitrage Optimization using QAOA in Cirq

---

## Introduction

This code shows how to use quantum computing, specifically the QAOA algorithm with Cirq, to identify profitable trading cycles in the foreign exchange market. These cycles take advantage of slight price differences between different currencies, which is known as currency arbitrage.

We turn this trading challenge into a math problem. Currencies become nodes in a network, and currency exchange rates become edges connecting those nodes. The main goal is to find a trade cycle that gives the most profit, following certain rules.


## Workflow Overview

![Workflow Diagram](https://i.ibb.co/N27zvzS2/Screenshot-2025-11-07-110838.png)

The workflow consists of:
1. **Problem Formulation** - Define the currency arbitrage problem as a QUBO
2. **QUBO to QAOA** - Translate the QUBO formulation to QAOA circuit
3. **Simulation** - Execute on Cirq simulator
4. **Analysis** - Evaluate results and validate implementation

---

## Model Setup

### Step 1: Setting Up the Problem

- Each possible currency is a node in the network.
- Every direct trade is an edge, and each edge has a value representing the exchange rate.
- We create a variable for each possible trade, which is 1 if that trade is used in the cycle, and 0 otherwise.

### Step 2: Objective and Constraints

- We want to maximize the profit by choosing the best sequence of trades.
- Using logarithms, the product of rates in the cycle becomes a sum, which is easier to optimize.
- We must follow two simple rules for a cycle:
    - Each currency can only have one incoming trade and one outgoing trade in the sequence.

### Step 3: Penalties for Invalid Trades

- If a cycle doesn't meet the rules, the algorithm adds a penalty to make those solutions undesirable.

### Step 4: Mapping to a Quantum Problem

- The whole setup becomes a binary optimization problem (QUBO), where we want the minimum "cost" (lowest energy) solution.
- This QUBO is converted into a format (Ising model) that works with quantum computers and Cirq.




# Problem Representation

#### Defining the Network

- Choose how many currencies to use (e.g., 3 for USD, EUR, GBP).
- Calculate how many binary decision variables are needed.

#### Qubit Assignment

- Each trade (edge) is represented by a unique qubit.

#### Setting Weights

- Assign exchange rates and compute the mathematical coefficients.

#### Adding Penalties

- Set up penalty weights to force the solution toward legal cycles only.

#### Transform to Ising Hamiltonian

- Convert the QUBO model into a list of quantum operations for the circuit.


## Mathematical Formulation

### Problem Definition

The currency arbitrage problem is modeled as finding an optimal cycle in a directed graph:

- Let $N$ be the set of currencies
- Let $r_{ij}$ be the exchange rate from currency $i$ to currency $j$
- Define decision variable $b_{ij} \in \{0, 1\}$ where:
  - $b_{ij} = 1$ if edge $(i, j)$ is part of the selected arbitrage cycle
  - $b_{ij} = 0$ otherwise

### Objective Function

The objective is to maximize the profit rate (product of exchange rates along the cycle). Using logarithms to convert the multiplicative objective to an additive one:

$$
C = \sum_{i,j} -\log(r_{ij})b_{ij}
$$

This function is **minimized** to maximize profit.

### Constraint Penalty Function

To ensure a valid arbitrage cycle (exactly one incoming and one outgoing edge per currency):

$$
P = \sum_{i} \left( \left(\sum_{j \neq i} b_{ji}\right) - 1 \right)^2 + \sum_{i} \left( \left(\sum_{j \neq i} b_{ij}\right) - 1 \right)^2
$$

Expanding this yields:

$$
P = \sum_{i} \sum_{j \neq j'} b_{ij}b_{ij'} + \sum_{j} \sum_{i \neq i'} b_{ij}b_{i'j}
$$

- **First term:** Ensures only one outgoing edge per currency
- **Second term:** Ensures only one incoming edge per currency

### Total Cost Function (QUBO)

$$
C_{tot} = C + m_p P
$$

where $m_p$ is a constraint factor (penalty weight).

### QUBO to Ising Transformation

For quantum annealing and gate-based quantum computing, we convert the QUBO to an Ising model using the mapping:

$$
b_i = \frac{1 - Z_i}{2}
$$

where $Z_i$ is the Pauli-Z operator on qubit $i$.

**Linear terms:** $c_k b_k = \frac{c_k}{2} - \frac{c_k}{2} Z_k$

**Quadratic terms:** $c_{kl} b_k b_l = \frac{c_{kl}}{4}(1 - Z_k - Z_l + Z_k Z_l)$

---

# 1. Install Necessary Libraries

In [ ]:
# Install Cirq for quantum circuit simulation
%pip install -q cirq

# 2. Problem Representation in Cirq


## 2.1 Define Problem Parameters and Qubits

In [22]:
import cirq
import numpy as np

# Define the number of currencies (nodes in the graph)
num_currencies = 3  # Example: USD, EUR, GBP

# Calculate the total number of decision variables (b_ij)
# For n currencies, there are n * (n - 1) possible directed edges
num_variables = num_currencies * (num_currencies - 1)

# Create a list of qubits

qubits = cirq.LineQubit.range(num_variables)

print(f"Number of currencies: {num_currencies}")
print(f"Number of decision variables (qubits): {num_variables}")
print(f"Qubits: {qubits}")

Number of currencies: 3
Number of decision variables (qubits): 6
Qubits: [cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5)]


## 2.2 Qubit-to-Edge Mapping

For a 3-currency example, we have 6 possible edges ($3 \times 2$):

| Qubit | Edge | Description |
|-------|------|-------------|
| 0 | $b_{01}$ | USD → EUR |
| 1 | $b_{02}$ | USD → GBP |
| 2 | $b_{10}$ | EUR → USD |
| 3 | $b_{12}$ | EUR → GBP |
| 4 | $b_{20}$ | GBP → USD |
| 5 | $b_{21}$ | GBP → EUR |

## 2.3 Define Exchange Rates and Linear Coefficients


In [23]:
# Placeholder exchange rates (using logarithmic scale)
# Symbolic linear coefficients representing -log(r_ij)
linear_coefficients = {
    qubits[0]: -np.log(1.1),    # USD → EUR
    qubits[1]: -np.log(1.3),    # USD → GBP
    qubits[2]: -np.log(1/1.1),  # EUR → USD
    qubits[3]: -np.log(1.2),    # EUR → GBP
    qubits[4]: -np.log(1/1.3),  # GBP → USD
    qubits[5]: -np.log(1/1.2)   # GBP → EUR
}

print("Linear coefficients (profit terms):")
for qubit, coeff in linear_coefficients.items():
    print(f"  {qubit}: {coeff:.6f}")

Linear coefficients (profit terms):
  q(0): -0.095310
  q(1): -0.262364
  q(2): 0.095310
  q(3): -0.182322
  q(4): 0.262364
  q(5): 0.182322


## 2.4 Define Penalty Terms (Quadratic Coefficients)

The penalty function ensures that each currency has exactly one incoming and one outgoing edge.

In [24]:
# Penalty factor (m_p)
penalty_factor = 5.0

quadratic_terms = {}

# Penalty for more than one outgoing edge from currency i
# For each currency i: sum_{j ≠ j'} b_ij * b_ij'

quadratic_terms[(qubits[0], qubits[1])] = penalty_factor
quadratic_terms[(qubits[2], qubits[3])] = penalty_factor
quadratic_terms[(qubits[4], qubits[5])] = penalty_factor

# Penalty for more than one incoming edge to currency j
# For each currency j: sum_{i ≠ i'} b_ij * b_i'j

quadratic_terms[(qubits[2], qubits[4])] = penalty_factor
quadratic_terms[(qubits[0], qubits[5])] = penalty_factor
quadratic_terms[(qubits[1], qubits[3])] = penalty_factor

# Linear terms from penalty function expansion: -2 * m_p * sum of all b_ij
# This comes from expanding (sum - 1)^2 = sum^2 - 2*sum + 1
# The -2*sum term contributes to linear coefficients
for qubit in qubits:
    linear_coefficients[qubit] = linear_coefficients.get(qubit, 0) - 4 * penalty_factor

# Constant term from penalty function: 2 * num_currencies * m_p
# This comes from the +1 terms in each (sum - 1)^2
constant_term = 2 * num_currencies * penalty_factor

print(f"\nPenalty factor: {penalty_factor}")
print(f"Number of quadratic terms: {len(quadratic_terms)}")
print(f"Constant term: {constant_term}")


Penalty factor: 5.0
Number of quadratic terms: 6
Constant term: 30.0


## 2.5 Convert QUBO to Ising Hamiltonian

We convert the QUBO formulation to an Ising Hamiltonian using the mapping $b_i = \frac{1 - Z_i}{2}$.

This transformation yields:
- **Ising linear terms:** $-\frac{c_k}{2} Z_k$
- **Ising quadratic terms:** $\frac{c_{kl}}{4} Z_k Z_l$
- **Constant offset:** Accumulates from the transformation

In [25]:
# Initialize Ising Hamiltonian terms
ising_linear_terms = {}
ising_quadratic_terms = {}
ising_constant_term = constant_term

# Convert linear terms:
for qubit, coeff in linear_coefficients.items():
    ising_constant_term += coeff / 2
    ising_linear_terms[qubit] = ising_linear_terms.get(qubit, 0) - coeff / 2

# Convert quadratic terms:
for (q1, q2), coeff in quadratic_terms.items():
    ising_constant_term += coeff / 4
    ising_linear_terms[q1] = ising_linear_terms.get(q1, 0) - coeff / 4
    ising_linear_terms[q2] = ising_linear_terms.get(q2, 0) - coeff / 4
    ising_quadratic_terms[(q1, q2)] = ising_quadratic_terms.get((q1, q2), 0) + coeff / 4

print("Ising Hamiltonian Terms:")
print(f"  Constant term: {ising_constant_term}")
print(f"  Number of linear Z terms: {len(ising_linear_terms)}")
print(f"  Number of quadratic ZZ terms: {len(ising_quadratic_terms)}")

Ising Hamiltonian Terms:
  Constant term: -22.5
  Number of linear Z terms: 6
  Number of quadratic ZZ terms: 6


## 2.6 Construct Hamiltonian for QAOA

We build a list of Hamiltonian terms (coefficient, operator pairs) for use in QAOA.

In [26]:
# Construct the Hamiltonian
hamiltonian_terms = []

# Add linear Z terms
for qubit, coeff in ising_linear_terms.items():
    if abs(coeff) > 1e-9:
        hamiltonian_terms.append((coeff, cirq.Z(qubit)))

# Add quadratic ZZ terms
for (q1, q2), coeff in ising_quadratic_terms.items():
    if abs(coeff) > 1e-9:
        hamiltonian_terms.append((coeff, cirq.Z(q1) * cirq.Z(q2)))

print(f"\nTotal Hamiltonian terms: {len(hamiltonian_terms)}")
print(f"  Linear terms: {len([t for t in hamiltonian_terms if isinstance(t[1], cirq.ops.PauliString) and len(t[1]) == 1])}")
print(f"  Quadratic terms: {len([t for t in hamiltonian_terms if isinstance(t[1], cirq.ops.PauliString) and len(t[1]) == 2])}")

print("\nExample Hamiltonian terms:")
for i, (coeff, op) in enumerate(hamiltonian_terms[:3]):
    print(f"  {coeff:.4f} * {op}")


Total Hamiltonian terms: 12
  Linear terms: 6
  Quadratic terms: 6

Example Hamiltonian terms:
  7.5477 * Z(q(0))
  7.6312 * Z(q(1))
  7.4523 * Z(q(2))


# 3. QAOA Implementation

The Quantum Approximate Optimization Algorithm (QAOA) consists of:
1. **Initial state preparation** - Uniform superposition
2. **Alternating layers** of:
   - Problem Hamiltonian evolution (parameterized by γ)
   - Mixing Hamiltonian evolution (parameterized by β)
3. **Measurement** in computational basis

## 3.1 Define QAOA Parameters

In [27]:
import sympy

# Define the number of QAOA layers (p)
p = 2

# Define variational parameters as symbolic variables

gammas = [sympy.Symbol(f'gamma_{i}') for i in range(p)]
betas = [sympy.Symbol(f'beta_{i}') for i in range(p)]

# For simulation, we need concrete values
# These are arbitrary initial values
initial_gamma_values = [0.1] * p
initial_beta_values = [0.1] * p

print(f"QAOA Parameters:")
print(f"  Number of layers (p): {p}")
print(f"  Gamma symbols: {gammas}")
print(f"  Beta symbols: {betas}")
print(f"  Initial gamma values: {initial_gamma_values}")
print(f"  Initial beta values: {initial_beta_values}")

QAOA Parameters:
  Number of layers (p): 2
  Gamma symbols: [gamma_0, gamma_1]
  Beta symbols: [beta_0, beta_1]
  Initial gamma values: [0.1, 0.1]
  Initial beta values: [0.1, 0.1]


## 3.2 Build QAOA Circuit Components


In [28]:
def initial_state_circuit(qubits):
    """
    Prepare initial state: uniform superposition over all basis states.
    Achieved by applying Hadamard gate to each qubit.
    """
    circuit = cirq.Circuit()
    circuit.append(cirq.H(q) for q in qubits)
    return circuit


def problem_hamiltonian_evolution(qubits, hamiltonian_terms, gamma):
    """
    Implement evolution under the problem Hamiltonian: exp(-i * gamma * H_problem).

    For Z terms: exp(-i * gamma * c * Z) ≈ Rz(-2 * gamma * c)
    For ZZ terms: exp(-i * gamma * c * Z_i * Z_j) implemented using ZZPowGate
    """
    circuit = cirq.Circuit()

    for coeff, op in hamiltonian_terms:
        # Single-qubit Z term
        if isinstance(op, cirq.ops.PauliString) and len(op) == 1:
            qubit = list(op.qubits)[0]
            circuit.append(cirq.Rz(rads=-2 * gamma * coeff)(qubit))

        # Two-qubit ZZ term
        elif isinstance(op, cirq.ops.PauliString) and len(op) == 2:
            q1, q2 = list(op.qubits)
            circuit.append(cirq.ZZPowGate(exponent=-2 * gamma * coeff / np.pi)(q1, q2))

    return circuit


def mixing_hamiltonian_evolution(qubits, beta):
    """
    Implement evolution under the mixing Hamiltonian: exp(-i * beta * sum_k X_k).

    This decomposes into: product_k exp(-i * beta * X_k)
    For each qubit: exp(-i * beta * X) ≈ Rx(2 * beta)
    """
    circuit = cirq.Circuit()
    circuit.append(cirq.Rx(rads=2 * beta)(q) for q in qubits)
    return circuit


def qaoa_circuit(qubits, hamiltonian_terms, gammas, betas, p):
    """
    Construct the full QAOA circuit with p layers.
    """
    circuit = initial_state_circuit(qubits)

    for i in range(p):
        circuit.append(problem_hamiltonian_evolution(qubits, hamiltonian_terms, gammas[i]))
        circuit.append(mixing_hamiltonian_evolution(qubits, betas[i]))

    return circuit


## 3.3 Construct and Display QAOA Circuit

In [29]:
# Build the QAOA circuit with symbolic parameters
qaoa_qc = qaoa_circuit(qubits, hamiltonian_terms, gammas, betas, p)

print("QAOA Circuit (with symbolic parameters):")
print(qaoa_qc)
print(f"\nCircuit depth: {len(qaoa_qc)}")
print(f"Total gates: {sum(1 for _ in qaoa_qc.all_operations())}")

QAOA Circuit (with symbolic parameters):
                                                                            ┌─────────────────────────────────────────────────────────────────────────────────────────────┐                                                                                    ┌─────────────────────────────────────────────────────────────────────────────────────────────┐
0: ───H───Rz(-15.0953101798043*gamma_0)───ZZ────────────────────────────────────────────────────────────────ZZ────────────────────────────────────────────────────────────────Rx(2*beta_0)───Rz(-15.0953101798043*gamma_1)───ZZ────────────────────────────────────────────────────────────────ZZ────────────────────────────────────────────────────────────────Rx(2*beta_1)───
                                          │                                                                 │                                                                                                                │                 

## 3.4 Resolve Parameters for Simulation

In [30]:
# Create parameter resolver with initial values
param_bindings = {gammas[i]: initial_gamma_values[i] for i in range(p)}
param_bindings.update({betas[i]: initial_beta_values[i] for i in range(p)})
param_resolver = cirq.ParamResolver(param_bindings)

# Resolve the symbolic parameters in the circuit
resolved_circuit = cirq.resolve_parameters(qaoa_qc, param_resolver)

print("Resolved QAOA Circuit (with concrete parameter values):")
print(resolved_circuit)

Resolved QAOA Circuit (with concrete parameter values):
                                   ┌────────────────────────┐                                         ┌────────────────────────┐
0: ───H───Rz(-0.48π)────ZZ──────────────────ZZ──────────────────Rx(0.064π)───Rz(-0.48π)────ZZ──────────────────ZZ──────────────────Rx(0.064π)───
                        │                   │                                              │                   │
1: ───H───Rz(-0.486π)───ZZ^-0.08────────────┼───────ZZ──────────Rx(0.064π)───Rz(-0.486π)───ZZ^-0.08────────────┼───────ZZ──────────Rx(0.064π)───
                                            │       │                                                          │       │
2: ───H───Rz(-0.474π)───ZZ──────────ZZ──────┼───────┼───────────Rx(0.064π)───Rz(-0.474π)───ZZ──────────ZZ──────┼───────┼───────────Rx(0.064π)───
                        │           │       │       │                                      │           │       │       │
3: ───H───Rz(-0.483π)───Z

# 4. Simulation and Testing

## 4.1 Run Simulation

In [31]:
# Instantiate Cirq simulator
simulator = cirq.Simulator()

repetitions = 1000

# Add measurement gates to the circuit
resolved_circuit_with_measurement = resolved_circuit + cirq.measure(*qubits, key='result')

# Run the simulation for 1000 repetitions
print(f"Running simulation with 1000 repetitions...")
results = simulator.run(resolved_circuit_with_measurement, repetitions=repetitions)

# Obtain measurement counts (histogram)
measurement_counts = results.histogram(key='result')

print(f"Number of unique bitstrings measured: {len(measurement_counts)}")
print(f"\nMeasurement counts (top 10 most frequent):")
for bitstring_int, count in measurement_counts.most_common(10):
    bitstring_bin = bin(bitstring_int)[2:].zfill(num_variables)
    print(f"  {bitstring_bin}: {count}")

Running simulation with 1000 repetitions...
Number of unique bitstrings measured: 64

Measurement counts (top 10 most frequent):
  111111: 79
  110111: 42
  111110: 42
  001111: 36
  111101: 34
  101111: 34
  111011: 33
  110011: 30
  001011: 29
  011111: 28


## 4.2 Identify Most Frequent Solution

In [32]:
# Get the most frequent bitstring
most_frequent_bitstring = measurement_counts.most_common(1)
most_frequent_bitstring_int = most_frequent_bitstring[0][0]
most_frequent_count = most_frequent_bitstring[0][1]

# Convert to binary string and tuple
bitstring_str = bin(most_frequent_bitstring_int)[2:].zfill(num_variables)
most_frequent_bitstring_tuple = tuple(int(bit) for bit in bitstring_str)

print(f"Most Frequent Solution:")
print(f"  Integer representation: {most_frequent_bitstring_int}")
print(f"  Binary string: {bitstring_str}")
print(f"  Tuple: {most_frequent_bitstring_tuple}")
print(f"  Count: {most_frequent_count} ({most_frequent_count/repetitions*100:.1f}%)")

Most Frequent Solution:
  Integer representation: 63
  Binary string: 111111
  Tuple: (1, 1, 1, 1, 1, 1)
  Count: 79 (7.9%)


## 4.3 Map Bitstring to Arbitrage Path

We interpret the bitstring as a set of selected edges and attempt to reconstruct the arbitrage cycle.

In [33]:
def map_bitstring_to_path(bitstring_tuple, num_currencies):
    """
    Map a bitstring to the corresponding currency arbitrage path.

    Returns:
        path_str: String representation of the path
        edges: List of (source, destination) edge tuples
    """
    num_variables = num_currencies * (num_currencies - 1)
    if len(bitstring_tuple) != num_variables:
        return "Invalid bitstring length", []

    # Extract edges where b_ij = 1
    edges = []
    k = 0
    for i in range(num_currencies):
        for j in range(num_currencies):
            if i != j:
                if bitstring_tuple[k] == 1:
                    edges.append((i, j))
                k += 1

    if not edges:
        return "No edges selected", []

    # Reconstruct path/cycle from selected edges
    path_nodes = []
    if edges:
        current_node = edges[0][0]
        path_nodes.append(current_node)

    visited_edges = set()
    remaining_edges = list(edges)

    # Follow edges to build the path
    while current_node is not None and len(visited_edges) < len(edges):
        found_next = False
        for start, end in remaining_edges:
            if (start, end) in visited_edges:
                continue
            if start == current_node:
                path_nodes.append(end)
                current_node = end
                visited_edges.add((start, end))
                found_next = True
                break
        if not found_next:
            if len(visited_edges) < len(edges):
                path_nodes.append("...")
            break

    path_str = " → ".join(map(str, path_nodes)) if path_nodes else "No path found"
    return path_str, edges


# Map the most frequent bitstring to a path
most_frequent_path_str, most_frequent_edges = map_bitstring_to_path(
    most_frequent_bitstring_tuple, num_currencies
)

print(f"\nArbitrage Path Interpretation:")
print(f"  Selected edges: {most_frequent_edges}")
print(f"  Path: {most_frequent_path_str}")


Arbitrage Path Interpretation:
  Selected edges: [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]
  Path: 0 → 1 → 0 → 2 → 0 → ...


## 4.4 Energy Calculation Functions


In [34]:
def calculate_qubo_energy(bitstring_tuple, linear_coefficients, quadratic_terms,
                         constant_term, qubits):
    """
    Calculate QUBO energy for a given bitstring.
    """
    qubit_values = {qubits[i]: bitstring_tuple[i] for i in range(len(qubits))}
    energy = constant_term

    # Linear terms
    for qubit, coeff in linear_coefficients.items():
        if qubit in qubit_values:
            energy += coeff * qubit_values[qubit]

    # Quadratic terms
    for (q1, q2), coeff in quadratic_terms.items():
        if q1 in qubit_values and q2 in qubit_values:
            energy += coeff * qubit_values[q1] * qubit_values[q2]

    return energy


def calculate_ising_energy(bitstring_tuple, ising_linear_terms, ising_quadratic_terms,
                          ising_constant_term, qubits):
    """
    Calculate Ising energy for a given bitstring.

    Mapping: b_k = 0 → Z_k = 1, b_k = 1 → Z_k = -1
    """
    qubit_z_values = {
        qubits[i]: (1 if bitstring_tuple[i] == 0 else -1)
        for i in range(len(qubits))
    }
    energy = ising_constant_term

    # Linear Z terms
    for qubit, coeff in ising_linear_terms.items():
        if qubit in qubit_z_values:
            energy += coeff * qubit_z_values[qubit]

    # Quadratic ZZ terms
    for (q1, q2), coeff in ising_quadratic_terms.items():
        if q1 in qubit_z_values and q2 in qubit_z_values:
            energy += coeff * qubit_z_values[q1] * qubit_z_values[q2]

    return energy


# Calculate energies for the most frequent bitstring
most_frequent_qubo_energy = calculate_qubo_energy(
    most_frequent_bitstring_tuple, linear_coefficients, quadratic_terms,
    constant_term, qubits
)

most_frequent_ising_energy = calculate_ising_energy(
    most_frequent_bitstring_tuple, ising_linear_terms, ising_quadratic_terms,
    ising_constant_term, qubits
)

print(f"\nEnergy of Most Frequent Bitstring:")
print(f"  QUBO energy: {most_frequent_qubo_energy:.4f}")
print(f"  Ising energy: {most_frequent_ising_energy:.4f}")


Energy of Most Frequent Bitstring:
  QUBO energy: -60.0000
  Ising energy: -60.0000


# 5. Results Analysis


## 5.1 Analyze Known Valid Cycles


In [35]:
# Define known valid cycles for 3 currencies

bitstring_cycle1 = (1, 0, 0, 1, 0, 1)
bitstring_cycle2 = (0, 1, 1, 0, 1, 0)

# Calculate energies for valid cycles
energy_cycle1 = calculate_qubo_energy(
    bitstring_cycle1, linear_coefficients, quadratic_terms, constant_term, qubits
)
energy_cycle2 = calculate_qubo_energy(
    bitstring_cycle2, linear_coefficients, quadratic_terms, constant_term, qubits
)

print("Valid Arbitrage Cycles:")
print(f"\nCycle 1: 0 → 1 → 2 → 0")
print(f"  Bitstring: {bitstring_cycle1}")
print(f"  QUBO energy: {energy_cycle1:.4f}")

print(f"\nCycle 2: 0 → 2 → 1 → 0")
print(f"  Bitstring: {bitstring_cycle2}")
print(f"  QUBO energy: {energy_cycle2:.4f}")

# Calculate energy for boundary cases
bitstring_all_zeros = (0,) * num_variables
energy_all_zeros = calculate_qubo_energy(
    bitstring_all_zeros, linear_coefficients, quadratic_terms, constant_term, qubits
)

bitstring_all_ones = (1,) * num_variables
energy_all_ones = calculate_qubo_energy(
    bitstring_all_ones, linear_coefficients, quadratic_terms, constant_term, qubits
)

print(f"\nBoundary Cases:")
print(f"  All zeros {bitstring_all_zeros}: {energy_all_zeros:.4f}")
print(f"  All ones {bitstring_all_ones}: {energy_all_ones:.4f}")

Valid Arbitrage Cycles:

Cycle 1: 0 → 1 → 2 → 0
  Bitstring: (1, 0, 0, 1, 0, 1)
  QUBO energy: -25.0953

Cycle 2: 0 → 2 → 1 → 0
  Bitstring: (0, 1, 1, 0, 1, 0)
  QUBO energy: -24.9047

Boundary Cases:
  All zeros (0, 0, 0, 0, 0, 0): 30.0000
  All ones (1, 1, 1, 1, 1, 1): -60.0000


## 5.2 Energy Distribution Analysis


In [36]:
def int_to_bitstring_tuple(n, num_bits):
    """Convert integer to bitstring tuple."""
    return tuple(int(bit) for bit in bin(n)[2:].zfill(num_bits))


# Calculate QUBO energy for all measured bitstrings
measured_bitstring_energies = {}
for bitstring_int, count in measurement_counts.items():
    bitstring_tuple = int_to_bitstring_tuple(bitstring_int, num_variables)
    energy = calculate_qubo_energy(
        bitstring_tuple, linear_coefficients, quadratic_terms, constant_term, qubits
    )
    measured_bitstring_energies[bitstring_tuple] = energy

# Sort by energy (lowest first)
sorted_measured_energies = sorted(
    measured_bitstring_energies.items(), key=lambda item: item[1]
)

print("Energy Distribution Analysis:")
print(f"\nTop 10 Lowest Energy Measured Bitstrings:")
print(f"{'Bitstring':<20} {'Energy':>10} {'Count':>8} {'Freq %':>8}")
print("-" * 50)

for bs_tuple, energy in sorted_measured_energies[:10]:
    bs_int = int("".join(map(str, bs_tuple)), 2)
    count = measurement_counts.get(bs_int, 0)
    freq_pct = (count / repetitions) * 100
    bs_str = "".join(map(str, bs_tuple))
    print(f"{bs_str:<20} {energy:>10.4f} {count:>8} {freq_pct:>7.1f}%")

Energy Distribution Analysis:

Top 10 Lowest Energy Measured Bitstrings:
Bitstring                Energy    Count   Freq %
--------------------------------------------------
111111                 -60.0000       79     7.9%
111101                 -50.2624       34     3.4%
111110                 -50.1823       42     4.2%
110111                 -50.0953       42     4.2%
011111                 -49.9047       28     2.8%
111011                 -49.8177       33     3.3%
101111                 -49.7376       34     3.4%
110110                 -40.2776       13     1.3%
011101                 -40.1671        9     0.9%
111001                 -40.0800       20     2.0%


## 5.3 Validate Most Frequent Solution

In [37]:
# Check if the most frequent bitstring is a valid cycle
def is_valid_cycle(edges, num_currencies):
    """
    Validate if a set of edges forms a valid arbitrage cycle.
    A valid cycle has exactly one incoming and one outgoing edge per currency.
    """
    if len(edges) != num_currencies:
        return False, "Incorrect number of edges"

    incoming_counts = {i: 0 for i in range(num_currencies)}
    outgoing_counts = {i: 0 for i in range(num_currencies)}

    for start, end in edges:
        outgoing_counts[start] += 1
        incoming_counts[end] += 1

    for i in range(num_currencies):
        if incoming_counts[i] != 1 or outgoing_counts[i] != 1:
            return False, f"Currency {i}: {incoming_counts[i]} in, {outgoing_counts[i]} out"

    return True, "Valid cycle"


is_valid, validation_msg = is_valid_cycle(most_frequent_edges, num_currencies)

print("\nValidation of Most Frequent Solution:")
print(f"  Bitstring: {most_frequent_bitstring_tuple}")
print(f"  Selected edges: {most_frequent_edges}")
print(f"  Number of edges: {len(most_frequent_edges)}")
print(f"  Is valid cycle? {is_valid}")
print(f"  Details: {validation_msg}")


Validation of Most Frequent Solution:
  Bitstring: (1, 1, 1, 1, 1, 1)
  Selected edges: [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]
  Number of edges: 6
  Is valid cycle? False
  Details: Incorrect number of edges


## 5.4 Compare Energies


In [38]:
print("\n" + "="*60)
print("ENERGY COMPARISON SUMMARY")
print("="*60)

comparison_data = [
    ("Most frequent bitstring", most_frequent_qubo_energy, most_frequent_count),
    ("Valid Cycle 1 (0→1→2→0)", energy_cycle1, measurement_counts.get(int("".join(map(str, bitstring_cycle1)), 2), 0)),
    ("Valid Cycle 2 (0→2→1→0)", energy_cycle2, measurement_counts.get(int("".join(map(str, bitstring_cycle2)), 2), 0)),
    ("All zeros (no edges)", energy_all_zeros, measurement_counts.get(0, 0)),
    ("All ones (all edges)", energy_all_ones, measurement_counts.get(2**num_variables - 1, 0))
]

print(f"\n{'Configuration':<30} {'Energy':>12} {'Count':>8} {'Freq %':>8}")
print("-" * 60)

for config, energy, count in comparison_data:
    freq_pct = (count / repetitions) * 100 if count > 0 else 0
    print(f"{config:<30} {energy:>12.4f} {count:>8} {freq_pct:>7.1f}%")

print("\n" + "="*60)


ENERGY COMPARISON SUMMARY

Configuration                        Energy    Count   Freq %
------------------------------------------------------------
Most frequent bitstring            -60.0000       79     7.9%
Valid Cycle 1 (0→1→2→0)            -25.0953        7     0.7%
Valid Cycle 2 (0→2→1→0)            -24.9047       14     1.4%
All zeros (no edges)                30.0000       15     1.5%
All ones (all edges)               -60.0000       79     7.9%

